## Creando Agentes del Servicio de Agentes de Azure AI con Semantic Kernel

![Flujo de Trabajo del Agente](https://github.com/kuljotSB/semantic-kernel/blob/main/Assets/Azure_AI_Agent.jpg?raw=true)


#### Instalando dependencias y bibliotecas

In [1]:
#%pip install semantic-kernel==1.28.1

In [2]:
#%pip install azure-identity

In [3]:
#%pip install azure-ai-projects==1.0.0b7

#### Creando nuestro Agente a través de código

In [1]:
# --- Importaciones Necesarias ---
from azure.identity import DefaultAzureCredential # Para la autenticación con Azure.
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings # La clase principal de Agente de Semantic Kernel para este flujo.
import os # Para leer variables de entorno.
from azure.ai.projects import AIProjectClient # El cliente de bajo nivel para interactuar con el servicio de Azure.
from dotenv import load_dotenv # Para cargar el archivo .env.
import asyncio # Para operaciones asíncronas.

# Se carga el archivo .env.
load_dotenv()

# Se obtiene el nombre del modelo a utilizar.
#model = os.getenv("AZURE_OPENAI_CHAT_COMPLETION_MODEL")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

# --- Creación del Cliente para el Servicio de Azure ---
# Se crea el cliente que se comunicará directamente con la plataforma de Agentes de Azure AI.
project_client = AzureAIAgent.create_client(credential=DefaultAzureCredential(),
                                          conn_str=os.getenv("AI_PROJECT_CONNECTION_STRING")
)

# --- Creación del Agente en Dos Pasos ---
# [INICIO create_agent]
# PASO 1: Crear la definición del agente en el servicio de Azure AI.
# Esta llamada a la API registra un nuevo agente en la nube de Azure.
agent_definition = await project_client.agents.create_agent( # añade get_agent(assistant_id="") para usar un agente ya creado
        model=deployment_name,
        name="my-assistant", # Nombre del agente en Azure.
        instructions="Eres un asistente muy útil", # Instrucciones base.
    )

# PASO 2: Crear un agente de Semantic Kernel que actúe como "control remoto" para el agente de Azure.
# Este es el objeto con el que interactuaremos en nuestro código.
agent = AzureAIAgent(
        client=project_client, # Se le pasa el cliente de bajo nivel para la comunicación.
        definition=agent_definition, # Se le pasa la definición del agente que creamos en la nube.
    )

#### Obteniendo una Respuesta de nuestro Agente

In [2]:
# Se define la pregunta del usuario.
user_input = "¿Cuál es la capital de Francia?"

# Se crea un hilo de conversación en el servicio de Azure. (Nota: no se usa en esta celda, pero se muestra cómo crearlo).
thread = project_client.agents.create_thread()


# Se define una función asíncrona para llamar al agente.
async def get_response_from_agent():
    # Se llama al método get_response de nuestro agente de Semantic Kernel.
    # Este método enviará el mensaje al agente alojado en Azure y esperará la respuesta.
    response =  await agent.get_response(
        messages = user_input, # El contenido del mensaje.
    )
    
    # Se devuelve la respuesta.
    return response

# Se ejecuta la función y se obtiene la respuesta.
response = await get_response_from_agent()
# Se imprime el contenido de la respuesta.
print(response)

La capital de Francia es París.


#### Viendo en Acción el Historial de Chat del Agente (Thread)

In [3]:
# Importa la clase específica para manejar hilos de conversación con este tipo de agente.
from semantic_kernel.agents import AzureAIAgentThread

# Se crea una instancia del hilo (thread). Este objeto mantendrá el historial de la conversación.
thread: AzureAIAgentThread = AzureAIAgentThread(client=project_client)

# Se inicializa una variable para controlar el bucle del chat.
continue_chat = True

# Comienza un bucle para chatear interactivamente.
while continue_chat:
    # Pide al usuario que ingrese su mensaje.
    user_input = input("Ingresa tu mensaje: ")
    # Si el usuario escribe 'exit', se termina la conversación.
    if user_input.lower() == "exit":
        continue_chat = False
        break
    # Se llama al agente, pasándole tanto el nuevo mensaje como el objeto 'thread'.
    # Al pasar el 'thread', el agente tendrá acceso a toda la conversación anterior para mantener el contexto.
    response = await agent.get_response(messages=user_input,
                                          thread = thread
    )
    
    # Se imprime la respuesta del agente en la consola.
    print(response)



C:\Users\raule\AppData\Local\Temp\ipykernel_24876\1715049901.py:5: RuntimeWarning: coroutine 'AgentsOperations.create_thread' was never awaited
  thread: AzureAIAgentThread = AzureAIAgentThread(client=project_client)


2 + 2 es igual a 4.
6 - 6 es igual a 0.
